**Loading Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency

In [2]:
aces = pd.read_csv('/content/drive/MyDrive/Northwell Pipelines/ACES Pipeline/10_average_graphs/picture_average_graph_features_aces.csv')
lpop = pd.read_csv('/content/drive/MyDrive/Northwell Pipelines/LPOP Pipeline/10_average_graphs/average_graph_features_lpop.csv')
remora = pd.read_csv('/content/drive/MyDrive/Northwell Pipelines/Remora Pipeline/10_average_graphs/BL_average_graph_features_remora.csv')

In [ ]:
df = pd.concat([aces,lpop,remora],axis=0, ignore_index=True)
df['uid'] = df['uid'].astype(str)
df_picture = df.loc[df['task'] == 'picture'].copy()
df_picture.drop(['Unnamed: 0', 'timepoint'], axis=1,inplace=True)

In [4]:
tlc_scores = ['tlc_01povspeech',
       'tlc_02povcontent', 'tlc_03pressure', 'tlc_04distract', 'tlc_05tangent',
       'tlc_06derail', 'tlc_07incoh', 'tlc_08illogic', 'tlc_09clang',
       'tlc_10neologism', 'tlc_11wordapprox', 'tlc_12circum', 'tlc_13lossgoal',
       'tlc_14persev', 'tlc_15echo', 'tlc_16block', 'tlc_17stilt',
       'tlc_18selfref', 'tlc_global', 'sans_6_vocalinf', 'sans_11_latency',
       'tlc_sum', 'tlc_d01povspeech', 'tlc_d02povcontent', 'tlc_d03pressure',
       'tlc_d04distract', 'tlc_d05tangent', 'tlc_d06derail', 'tlc_d07incoh',
       'tlc_d08illogic', 'tlc_d09clang', 'tlc_d10neologism',
       'tlc_d11wordapprox', 'tlc_d12circum', 'tlc_d13lossgoal',
       'tlc_d14persev', 'tlc_d15echo', 'tlc_d16block', 'tlc_d17stilt',
       'tlc_d18selfref', 'tlc_dglobal']

tlc_factors = ['tlc_3f_inefficient', 'tlc_3f_incoherent', 'tlc_3f_impexpress']

sans = ['sans_global_asocanhed', 'sans_global_affect', 'sans_global_alogia', 'sans_global_avolition', 'sans_tot_asociality', 'sans_tot_global', 'sans_total']

bprs = ['bprs_total','bprs_factor_anxdep' ,'bprs_factor_hostsus' ,'bprs_factor_thought', 'bprs_factor_withret']

demo = ['demo_age','demo_sex', 'demo_gender', 'demo_race','demo_education','demo_caregiver_education']

In [ ]:
clinical =  pd.read_csv('/content/drive/MyDrive/Northwell Pipelines/Shared Pipeline Files/crossdx_clin.csv')
clinical.columns.tolist()

In [ ]:
clinical =  pd.read_csv('/content/drive/MyDrive/Northwell Pipelines/Shared Pipeline Files/crossdx_clin.csv')

for index,row in clinical.iterrows():
  if row['study'] == 'Remora':
    clinical.at[index,'grid'] = row['grid'] + '_BL_2'

clinical['uid'] = clinical['grid'].astype(str)
clinical = clinical[['uid','study','anypsychosis','group'] +demo+ tlc_factors + tlc_scores + sans + bprs + ['qls_interpersonal','qls_role']]

clinical

In [ ]:
df_picture_graph_clinical = df_picture.merge(clinical, on = 'uid')
df_picture_graph_clinical.to_csv('/content/drive/MyDrive/Northwell Pipelines/Statistics/dataframes/picture_graph_tlc.csv')

**Table 1. Demographic Characteristics of Participants**

In [ ]:
### Number of participants
len(df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]) # =124
len(df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]) # =81

In [ ]:
### Age
## Psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_age'].mean() # =25.9
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_age'].std() # =6.3
## Non-psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_age'].mean() # =25.2
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_age'].std() # =5.9
## Association Test
U1, p = mannwhitneyu(df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_age'].dropna(), df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_age'].dropna())
p # = 0.20

In [22]:
### Sex
chisqt = pd.crosstab(df_picture_graph_clinical.anypsychosis, df_picture_graph_clinical.demo_sex, margins=True)
value = np.array([chisqt.iloc[0][0:2].values, chisqt.iloc[1][0:2].values])
print(chi2_contingency(value)[0:3])
chisqt

(10.339655052640776, 0.0013020234196261813, 1)


demo_sex,Female,Male,All
anypsychosis,,,
0,83,41,124
1,35,46,81
All,118,87,205


In [24]:
###Race
for index, row in df_picture_graph_clinical.iterrows():
  if row['demo_race'] == 'Black':
    df_picture_graph_clinical.at[index,'race'] = 'AfricanAmerican'
  elif row['demo_race'] == 'White':
    df_picture_graph_clinical.at[index,'race'] = 'Caucasian'
  elif row['demo_race'] == 'Asian':
    df_picture_graph_clinical.at[index,'race'] = 'Asian'
  else:
    df_picture_graph_clinical.at[index,'race'] = 'Other'

chisqt = pd.crosstab(df_picture_graph_clinical.anypsychosis, df_picture_graph_clinical.race, margins=True)

value = np.array([chisqt.iloc[0][0:4].values,
                  chisqt.iloc[1][0:4].values])
print(chi2_contingency(value)[0:3])
chisqt

(20.584285034329238, 0.00012841832263632482, 3)


race,AfricanAmerican,Asian,Caucasian,Other,All
anypsychosis,,,,,
0,22,13,69,20,124
1,31,11,20,19,81
All,53,24,89,39,205


In [ ]:
### Education
## Psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_education'].mean() # =14.1
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_education'].std() # =2
## Non-psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_education'].mean() # =15.5
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_education'].std() # =1.9
## Association Test
U1, p = mannwhitneyu(df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_education'].dropna(), df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_education'].dropna())
p # =2.2706093583006617e-06

In [ ]:
### Caregiver Education
## Psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_caregiver_education'].mean() # =14.6
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_caregiver_education'].std() # =3
## Non-psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_caregiver_education'].mean() # =15.3
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_caregiver_education'].std() # =2.3
## Association Test
U1, p = mannwhitneyu(df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['demo_caregiver_education'].dropna(), df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['demo_caregiver_education'].dropna())
p # = 0.07

In [ ]:
### BPRS
## Psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['bprs_total'].mean() # =42
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['bprs_total'].std() # =14
## Non-psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['bprs_total'].mean() # =22
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['bprs_total'].std() # =4
## Association Test
U1, p = mannwhitneyu(df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['bprs_total'].dropna(), df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['bprs_total'].dropna())
p # =3.86719045508284e-14

In [ ]:
### SANS
## Psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['sans_total'].mean() # =30
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['sans_total'].std() # =15
## Non-psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['sans_total'].mean() # =4
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['sans_total'].std() # =4.5
## Association Test
U1, p = mannwhitneyu(df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['sans_total'].dropna(), df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['sans_total'].dropna())
p # = 4.045719075117391e-16

In [ ]:
### TLC
## Psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['tlc_sum'].mean() # =16
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['tlc_sum'].std() # =14
## Non-psychosis
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['tlc_sum'].mean() # =4
df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['tlc_sum'].std() # =4.5
## Association Test
U1, p = mannwhitneyu(df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 1]['tlc_sum'].dropna(), df_picture_graph_clinical.loc[df_picture_graph_clinical['anypsychosis'] == 0]['tlc_sum'].dropna())
p # = 1.775633732628493e-14